In [1]:
import numpy as np
import torch
import pandas as pd

# # sbi
import sbi.utils as utils
from sbi.utils.get_nn_models import posterior_nn
from sbi.inference.base import infer
from sbi.inference import SNPE

# import saver utilities
import pickle

# to check how long things take
import time

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


## Loading observed experimental data

In [2]:
# All 25 degree Celcius mouse motor cortex (M1) electrophysiological data, preprocessed
M1_25degree = pickle.load(open('pickles/M1_features.pickle', 'rb'))
ephys_features = M1_25degree['X_o'].columns
Xo = M1_25degree['X_o'].copy()

In [3]:
prop = pd.read_csv('../data/m1_patchseq_meta_data.csv', sep = '\t')
prop = prop.rename(columns = {'Targeted layer': 'Layer'})
prop = prop[['Cell', 'Layer', 'Cre', 'RNA type']]
prop = prop.set_index('Cell')
prop=prop.reindex(Xo.index)
no_low_qual=np.array(list(map(str,prop['RNA type'].values)))!='nan'
prop=prop.loc[no_low_qual,:]
Xo = Xo.loc[no_low_qual,:]
celltypes=prop['RNA type']

## The uniform prior

In [4]:
model_param_names = np.array(['C', r'$R_{input}$', r'$\tau$', r'$g_{Nat}$', r'$g_{Na}$', r'$g_{Kd}$', r'$g_{M}$',
                         r'$g_{Kv31}$', r'$g_{L}$', r'$E_{leak}$', r'$\tau_{max}$', 'VT', 'rate_to_SS_factor'])
prior_min = [0.1,  20,  0.1,    0,        0,      0,      0,      0,      0, -130,    50,    -90,   0.1]
prior_max = [15,   1000,   70,   250,     100,      30,    3,     250,     3,  -50,  4000,   -35,    3]
prior = utils.torchutils.BoxUniform(
    low=torch.as_tensor(prior_min),
    high=torch.as_tensor(prior_max)
)

## Training schedules

In [5]:
feature_list=range(23) # we consider all ephys features
N_closest=1000 # number of closest prior simulations per cell we are going to keep

In [6]:
theta=np.load('save_sims/M1_chunks/full_batch.npz')['theta']
stats=np.load('save_sims/M1_chunks/full_batch.npz')['stats']

# Do not include non-spiking simulations
keeping=(~np.isnan(np.mean(stats, axis=1)))&(~np.isinf(np.mean(stats, axis=1)))
stats = stats[keeping,:]

tcmalloc: large alloc 1380007936 bytes == 0x3674a000 @ 


In [7]:
stats.shape

(7155552, 23)

In [8]:
all_stats=[]
all_theta=[]

theta=np.load('./save_sims/M1_chunks/full_batch.npz')['theta']
stats=np.load('./save_sims/M1_chunks/full_batch.npz')['stats']

# Do not include non-spiking simulations
keeping=(~np.isnan(np.mean(stats, axis=1)))&(~np.isinf(np.mean(stats, axis=1)))
stats = stats[keeping,:]

# Mean and standard deviation to Z-score with
s_mean=stats.mean(axis=0)
s_std=stats.std(axis=0)

t = time.time()

for i, cell_name in enumerate(Xo.index): # iterate over every cell in our experimental dataset
    print('.',end='')

    # Pick N_closest simulations to the experimental observation's summary statistics
    # Distance is the Euclidean Z-scored distance
    xo=Xo.loc[cell_name,:].values[:-4]

    ind=np.argsort(
        np.sum(
            np.square(
                (stats-s_mean)/s_std-(xo-s_mean)/s_std
            ), axis=1
        )
    )[0:N_closest]
    
    stats_=stats[ind,:]
    theta_=theta[keeping,:][ind,:]
    
    all_stats.append(stats_)
    all_theta.append(theta_)
            
t = time.time() - t
m,s = divmod(t, 60)
h,m = divmod(m, 60)
print('\nTime: {}h {:2.0f}m {:2.0f}s'.format(h,m,s))

tcmalloc: large alloc 1380007936 bytes == 0xb46ba000 @ 


...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
Time: 0.0h 39m 56s


In [9]:
theta_tr=all_theta[0]
stats_tr=all_stats[0]

for i in range(len(all_theta)-1):
    theta_tr=np.concatenate((theta_tr, all_theta[i+1]))
    stats_tr=np.concatenate((stats_tr, all_stats[i+1]))

np.savez('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest),
     theta=theta_tr,
     stats=stats_tr
    )

In [10]:
training_schedules={'0':{}, '1':{}, '2a':{}, '2b':{}, '2c':{}, '2d':{}, '2e':{}, '3':{}, '4':{}}

#### Training schedule 0: a batch of spiking non-inf/non-nan simulations from the prior.

In [11]:
theta=np.load('./save_sims/M1_chunks/full_batch.npz')['theta']
stats=np.load('./save_sims/M1_chunks/full_batch.npz')['stats']

## Do not include non-spiking simulations
keeping=(~np.isnan(np.mean(stats, axis=1)))&(~np.isinf(np.mean(stats, axis=1))) # delete Nan simulations
theta_tr = theta[keeping,:][::43,:]
stats_tr = stats[keeping,:][::43,:]

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['0'].update({'theta':theta_tr, 'stats':stats_tr})

tcmalloc: large alloc 1380007936 bytes == 0x155270000 @ 


(# sims, # model parameters):  (166409, 13)
(# sims, # ephys features):  (166409, 23)


#### Training schedule 1: *unique* best Euclidean prior simulations.

Some of our experimental observations have very simimlar observed ephys features or summary statistics, so they can share prior simulations in their batch of N_closest best Euclidean prior simulations. After concatenating all batches of prior simulations (every batch of prior simulations belongs to one experimental observation), there exist duplicates of prior simulations. We therefore keep only unique ones as to avoid duplicate training examples.

In [12]:
theta_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['theta']
stats_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['stats']
_, unique_tr_ids=np.unique(np.sum(stats_tr, axis=1),return_index=True)
theta_tr=theta_tr[unique_tr_ids,:]
stats_tr=stats_tr[unique_tr_ids,:]

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['1'].update({'theta':theta_tr, 'stats':stats_tr})

(# sims, # model parameters):  (164915, 13)
(# sims, # ephys features):  (164915, 23)


#### Training schedule 2a: noise with 0.001 amplitude added to ephys features of unique best Euclidean prior simulations

In [13]:
theta_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['theta']
stats_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['stats']
_, unique_tr_ids=np.unique(np.sum(stats_tr, axis=1),return_index=True)
theta_tr=theta_tr[unique_tr_ids,:]
stats_tr=stats_tr[unique_tr_ids,:]
eps=0.001 # noise amplitude
noise_stats=np.random.multivariate_normal(
    np.zeros((stats_tr.shape[1])),
    eps*np.diag(stats_tr.std(axis=0)),
    size=stats_tr.shape[0]
)
stats_tr += noise_stats

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['2a'].update({'theta':theta_tr, 'stats':stats_tr})

(# sims, # model parameters):  (164915, 13)
(# sims, # ephys features):  (164915, 23)


#### Training schedule 2b: noise with 0.01 amplitude added to ephys features of unique best Euclidean prior simulations

In [14]:
theta_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['theta']
stats_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['stats']
_, unique_tr_ids=np.unique(np.sum(stats_tr, axis=1),return_index=True)
theta_tr=theta_tr[unique_tr_ids,:]
stats_tr=stats_tr[unique_tr_ids,:]
eps=0.01 # noise amplitude
noise_stats=np.random.multivariate_normal(
    np.zeros((stats_tr.shape[1])),
    eps*np.diag(stats_tr.std(axis=0)),
    size=stats_tr.shape[0]
)
stats_tr += noise_stats

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['2b'].update({'theta':theta_tr, 'stats':stats_tr})

(# sims, # model parameters):  (164915, 13)
(# sims, # ephys features):  (164915, 23)


#### Training schedule 2c: noise with 0.05 amplitude added to ephys features of unique best Euclidean prior simulations

In [15]:
theta_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['theta']
stats_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['stats']
_, unique_tr_ids=np.unique(np.sum(stats_tr, axis=1),return_index=True)
theta_tr=theta_tr[unique_tr_ids,:]
stats_tr=stats_tr[unique_tr_ids,:]
eps=0.05 # noise amplitude
noise_stats=np.random.multivariate_normal(
    np.zeros((stats_tr.shape[1])),
    eps*np.diag(stats_tr.std(axis=0)),
    size=stats_tr.shape[0]
)
stats_tr += noise_stats

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['2c'].update({'theta':theta_tr, 'stats':stats_tr})

(# sims, # model parameters):  (164915, 13)
(# sims, # ephys features):  (164915, 23)


#### Training schedule 2d: noise with 0.1 amplitude added to ephys features of unique best Euclidean prior simulations

In [16]:
theta_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['theta']
stats_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['stats']
_, unique_tr_ids=np.unique(np.sum(stats_tr, axis=1),return_index=True)
theta_tr=theta_tr[unique_tr_ids,:]
stats_tr=stats_tr[unique_tr_ids,:]
eps=0.1 # noise amplitude
noise_stats=np.random.multivariate_normal(
    np.zeros((stats_tr.shape[1])),
    eps*np.diag(stats_tr.std(axis=0)),
    size=stats_tr.shape[0]
)
stats_tr += noise_stats

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['2d'].update({'theta':theta_tr, 'stats':stats_tr})

(# sims, # model parameters):  (164915, 13)
(# sims, # ephys features):  (164915, 23)


#### Training schedule 2e: noise with 1 amplitude added to ephys features of unique best Euclidean prior simulations

In [17]:
theta_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['theta']
stats_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['stats']
_, unique_tr_ids=np.unique(np.sum(stats_tr, axis=1),return_index=True)
theta_tr=theta_tr[unique_tr_ids,:]
stats_tr=stats_tr[unique_tr_ids,:]
eps=1 # noise amplitude
noise_stats=np.random.multivariate_normal(
    np.zeros((stats_tr.shape[1])),
    eps*np.diag(stats_tr.std(axis=0)),
    size=stats_tr.shape[0]
)
stats_tr += noise_stats

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['2e'].update({'theta':theta_tr, 'stats':stats_tr})

(# sims, # model parameters):  (164915, 13)
(# sims, # ephys features):  (164915, 23)


#### Training schedule 3: noise with 0.05 amplitude added to ephys features *and* model parameters of unique best Euclidean prior simulations

In [18]:
theta_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['theta']
stats_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['stats']
_, unique_tr_ids=np.unique(np.sum(stats_tr, axis=1),return_index=True)
theta_tr=theta_tr[unique_tr_ids,:]
stats_tr=stats_tr[unique_tr_ids,:]
eps=0.05 # noise amplitude
noise_stats=np.random.multivariate_normal(
    np.zeros((stats_tr.shape[1])),
    eps*np.diag(stats_tr.std(axis=0)),
    size=stats_tr.shape[0]
)
noise_params=np.random.multivariate_normal(
    np.zeros((theta_tr.shape[1])),
    eps*np.diag(theta_tr.std(axis=0)),
    size=theta_tr.shape[0]
)
stats_tr += noise_stats
theta_tr += noise_params

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['3'].update({'theta':theta_tr, 'stats':stats_tr})

(# sims, # model parameters):  (164915, 13)
(# sims, # ephys features):  (164915, 23)


#### Training schedule 4: noise with 0.05 amplitude added to ephys features of less *non-unique* best Euclidean prior simulations + original less best unique Euclidean prior simulations themselves

We would like to approximately have a same number of training examples for SBI across training schedules, so that the difference in performance cannot be due to significant different training set sizes, but due to our different training schedules used. To this end, we will start from a reduced batch of best Euclidean prior simulations per experimental observation and then do data augmentation.

In [19]:
theta_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['theta']
stats_tr = np.load('./save_sims/best_{}_Euclidean_sims.npz'.format(N_closest))['stats']
theta_tr=np.reshape(theta_tr, (N_closest, Xo.shape[0], len(model_param_names)))
stats_tr=np.reshape(stats_tr, (N_closest, Xo.shape[0], len(ephys_features[:-4])))
N_closest_new=69
theta_tr=theta_tr[:N_closest_new,:,:]
stats_tr=stats_tr[:N_closest_new,:,:]
theta_tr=np.reshape(theta_tr, (N_closest_new*Xo.shape[0], len(model_param_names)))
stats_tr=np.reshape(stats_tr, (N_closest_new*Xo.shape[0], len(ephys_features[:-4])))
eps=0.05 # noise amplitude
noise_stats=np.random.multivariate_normal(np.zeros((stats_tr.shape[1])),
                                    eps*np.diag(stats_tr.std(axis=0)),
                                    size=2*stats_tr.shape[0])
theta_tr=np.concatenate(
    (theta_tr,
     theta_tr,
     theta_tr
    ),
    axis=0
)

stats_tr=np.concatenate(
    (stats_tr,
     stats_tr+noise_stats[:stats_tr.shape[0],:],
     stats_tr+noise_stats[stats_tr.shape[0]:2*stats_tr.shape[0],:]
    ),
    axis=0
)

_, unique_tr_ids=np.unique(np.sum(stats_tr, axis=1),return_index=True)
theta_tr=theta_tr[unique_tr_ids,:]
stats_tr=stats_tr[unique_tr_ids,:]

print('(# sims, # model parameters): ', theta_tr.shape)
print('(# sims, # ephys features): ', stats_tr.shape)

training_schedules['4'].update({'theta':theta_tr, 'stats':stats_tr})

(# sims, # model parameters):  (167134, 13)
(# sims, # ephys features):  (167134, 23)


## Building amortized posteriors

In [ ]:
print('Building amortized posterior with training schedules: ')

t = time.time()

for tr_schedule in training_schedules:
    print('\n', tr_schedule)
    density_estimator_build_fun = posterior_nn(model='maf')
    inference = SNPE(
        prior, 
        density_estimator=density_estimator_build_fun
    )
    _ = inference.append_simulations(
                 torch.as_tensor(training_schedules[tr_schedule]['theta'], dtype=torch.float32),
                 torch.as_tensor(training_schedules[tr_schedule]['stats'], dtype=torch.float32)
                ).train()
    posterior = inference.build_posterior()

    with open('save_posteriors/training_schedule_{}.pickle'.format(tr_schedule), 'wb') as f:
        pickle.dump(posterior, f)
    
t = time.time() - t
m,s = divmod(t, 60)
h,m = divmod(m, 60)
print('\nTime: {}h {:2.0f}m {:2.0f}s'.format(h,m,s))

Building amortized posterior with training schedules: 

 0
 Neural network successfully converged after 206 epochs.
 1
 Neural network successfully converged after 314 epochs.
 2a
 Training neural network. Epochs trained: 157

#### Save 10 random posterior samples and the highest posterior sample for each observed experimental cell for each amortized posterior.

In [21]:
THETAS={
    '0':{'highest posterior samples':{}, '10 random samples':{}},
    '1':{'highest posterior samples':{}, '10 random samples':{}},
    '2a':{'highest posterior samples':{}, '10 random samples':{}},
    '2b':{'highest posterior samples':{}, '10 random samples':{}},
    '2c':{'highest posterior samples':{}, '10 random samples':{}},
    '2d':{'highest posterior samples':{}, '10 random samples':{}},
    '2e':{'highest posterior samples':{}, '10 random samples':{}},
    '3':{'highest posterior samples':{}, '10 random samples':{}},
    '4':{'highest posterior samples':{}, '10 random samples':{}}
}

For some experiments, depending on the training schedule, its summary statistics could be so different from simulations, the posterior finds barely any support from the prior. Sampling will therefore take prohibitively long. We therefore check and put them in the 'fails' dictionary.  

In [36]:
fails={'4':[66, 446, 532, 856], '3':[], '2e':[], '2d':[], '2c':[856], '2b':[101], '2a':[], '1':[], '0':[]}

In [ ]:
print('Extracting the highest posterior sample and 10 random posterior samples', 
      'for each experimentally observed cell and for each training schedule: ')
index=0
for tr_schedule in ['0', '1', '2a', '2b', '2c', '2d', '2e', '3', '4']:
    print('\n', 'Training schedule: ', tr_schedule)
    
    with open('save_posteriors/training_schedule_{}.pickle'.format(tr_schedule), 'rb') as f:
        posterior=pickle.load(f)
        
    for i in range(index, Xo.shape[0]):
        #if i not in fails[tr_schedule]:
        xo=Xo.iloc[i,:].values
        cell_name=Xo.index[i]
        print(str(i) + ' ', end='')

        # sampling 10000 from the posterior
        samples=posterior.sample(
            (10000,),
            x=torch.as_tensor(xo[feature_list], dtype=float),
            show_progress_bars=False
        )

        # highest posterior simulation
        highest_post_sample_1=samples[np.argsort(np.array(posterior.log_prob(theta=samples, x=xo[feature_list])))[-1],:]

        THETAS[tr_schedule]['highest posterior samples'].update({cell_name:np.array(highest_post_sample_1)})
        THETAS[tr_schedule]['10 random samples'].update({cell_name:samples[torch.randint(10000, (10,)),:]})
        # else:
        #     THETAS[tr_schedule]['highest posterior samples'].update({cell_name:np.ones((13,))*np.nan})
        #     THETAS[tr_schedule]['10 random samples'].update({cell_name:torch.ones((10,13))*torch.nan})            
        #     continue
        
with open('save_model_parameters/across_training_schedules.pickle', 'wb') as f:
    pickle.dump(THETAS, f)

Extracting the highest posterior sample and 10 random posterior samples for each experimentally observed cell and for each training schedule: 

 Training schedule:  0
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 